In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from datetime import datetime as dt
from datetime import timedelta
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [7]:
rpath = r"/Users/bashirulazambiswas/Documents/Sadia/Research/CARES homeless project/Code/2023/2023_SNAM_Trace/2023_Git/TRACE/Result"

In [13]:
def M_exit(k):
    pred_1e = pd.read_csv(rpath+"//ToyDataset_Prediction - M_exit exit points.csv").drop(['Unnamed: 0'],axis=1)
    pred_1e['ClientID'] = pred_1e['ClientID'].astype(str)
    pred_1e['ClientID'] = [pred_1e['ClientID'].iloc[i]+'_e' for i in range(len(pred_1e['ClientID']))]
    pred_1i = pd.read_csv(rpath+"//ToyDataset_Prediction - M_exit interim points.csv").drop(['Unnamed: 0'],axis=1)
    pred_1i['ClientID'] = pred_1i['ClientID'].astype(str)
    pred_1i['ClientID'] = [pred_1i['ClientID'].iloc[i]+'_i' for i in range(len(pred_1i['ClientID']))]
    pred_1 = pd.concat([pred_1e,pred_1i])
    prediction = pd.DataFrame()
    prediction['actual'] = pred_1['actual']
    for i in range(1,k+1):
        prediction['Prediction@'+str(i)] = pred_1['Prediction@'+str(i)]    
    return prediction

def M_int(k):
    #TRACE2
    pred_2e = pd.read_csv(rpath+"//ToyDataset_Prediction - M_int exit points.csv").drop(['Unnamed: 0'],axis=1)
    pred_2e['ClientID'] = pred_2e['ClientID'].astype(str)
    pred_2e['ClientID'] = [pred_2e['ClientID'].iloc[i]+'_e' for i in range(len(pred_2e['ClientID']))]
    pred_2i = pd.read_csv(rpath+"//ToyDataset_Prediction - M_int interim points.csv").drop(['Unnamed: 0'],axis=1)
    pred_2i['ClientID'] = pred_2i['ClientID'].astype(str)
    pred_2i['ClientID'] = [pred_2i['ClientID'].iloc[i]+'_i' for i in range(len(pred_2i['ClientID']))]
    pred_2 = pd.concat([pred_2e,pred_2i])
    prediction = pd.DataFrame()
    prediction['actual'] = pred_2['actual']
    for i in range(1,k+1):
        prediction['Prediction@'+str(i)] = pred_2['Prediction@'+str(i)]
    return prediction

def accuracy(pred,actual):
    matched = len(pred[pred==actual])
    accuracy = matched/len(pred)
    return accuracy


def get_stacking(k,models): #Prepares the input and output for the meta model of each meta model
    prediction_M_int = M_int(k)
    prediction_M_exit = M_exit(k)
    
    prediction = pd.DataFrame()
    #TRACE1 inputs
    for model in models:
        for i in range(1,k+1):
            if model == "M_exit":
                prediction_temp = pd.get_dummies(prediction_M_exit['Prediction@'+str(i)],prefix=model+"@"+str(i))
            elif model == "M_int":
                prediction_temp = pd.get_dummies(prediction_M_int['Prediction@'+str(i)],prefix=model+"@"+str(i))
                
            prediction = pd.concat([prediction,prediction_temp],axis=1)
    prediction['actual'] = prediction_M_int['actual']
    
    return prediction

        
def shuffle_prediction(prediction):
    shuffled_prediction = shuffle(prediction).reset_index(drop=True)    
    return shuffled_prediction


def train_test_split(prediction,test_size=0.25,CV=False,i = 0,N=0):
    n = int(len(prediction)*(1-test_size))
    
    if CV:
        X_test = prediction.drop(['actual'],axis=1).iloc[i:i+N]
        y_test = prediction['actual'].iloc[i:i+N]
        temp = prediction.drop(X_test.index)
        X_train = temp.drop(['actual'],axis=1)
        y_train = temp['actual']
    else:
        n = int(len(prediction)*(1-test_size))
        X_train = prediction.drop(['actual'],axis=1).iloc[0:n]
        y_train = prediction['actual'].iloc[0:n]
        X_test = prediction.drop(['actual'],axis=1).iloc[n::]
        y_test = prediction['actual'].iloc[n::]
    
    return X_train,y_train,X_test,y_test

def meta_model(X_train,y_train):
    model = LogisticRegression(max_iter=500).fit(X_train,y_train)
#     model = RandomForestClassifier().fit(X_train,y_train)
#     model = SVC().fit(X_train,y_train)

    return model

def trainTestData(X_train,X_test,k,model):
    Xtrain_TRACE = pd.DataFrame() 
    Xtest_TRACE = pd.DataFrame()
    for col in X_train.columns:
        for i in range(1,k+1):
            if model+"@"+str(i) in col:
                Xtrain_TRACE[col] = X_train[col]
                Xtest_TRACE[col] = X_test[col]
#                 Xval_TRACE[col] = X_val[col]
    return Xtrain_TRACE,Xtest_TRACE 



        
def accuracyAtk(k,model):
    if model == "M_exit":
        prediction = M_exit(k)
    elif model == "M_int":
        prediction = M_int(k) 
    prediction_copy = prediction.reset_index(drop=True)
    precision_df = pd.DataFrame()
    accuracy_score = pd.DataFrame()
    for i in range(1,k+1):
        temp = prediction_copy[prediction_copy['Prediction@'+str(i)]==prediction_copy['actual']][['Prediction@'+str(i),'actual']]
        precision_df = precision_df.append(temp)
        prediction_copy = prediction_copy.drop(temp.index)
        accuracy_score = accuracy_score.append({"k":i,"accuracy":len(precision_df)/len(prediction)},ignore_index=True)
    return accuracy_score

def classwise_recall(k,CV,prediction,actual,model):
    recall_df = pd.DataFrame()
    for p in np.unique(actual):
        TP = prediction[(actual==p) & (prediction==p)]
        FN = prediction[(actual==p) & (prediction!=p)]
        recall = len(TP)/(len(TP)+len(FN))
        recall_df = recall_df.append({"k":k,"CV":CV,"p":p,"recall":recall,'model':model},ignore_index=True)
    return recall_df
    
    


In [22]:
###### WITH CROSS VALIDATION #######
# data prep
scores = pd.DataFrame()
meta_recall_M_exit = pd.DataFrame()
meta_recall_M_int = pd.DataFrame()
meta2tier_recall = pd.DataFrame()

meta_precision_M_exit = pd.DataFrame()
meta_precision_M_int = pd.DataFrame()
meta2tier_precision = pd.DataFrame()

model_exit = 'M_exit'
model_int = 'M_int'

for k in range(1,11):
    prediction = get_stacking(k,[model_exit,model_int])
    CV = 5
    N = int(len(prediction)/CV)
    a = 0
    for i in range(0,len(prediction),N):
        a = a+1
        X_train,y_train,X_test,y_test = train_test_split(prediction,CV=False,i=i,N=N)
       # M_exit 
        Xtrain_M_exit,Xtest_M_exit = trainTestData(X_train,X_test,k,model_exit)
        meta_model_M_exit = meta_model(Xtrain_M_exit,y_train)
        train_prediction_M_exit = meta_model_M_exit.predict(Xtrain_M_exit)
        test_prediction_M_exit = meta_model_M_exit.predict(Xtest_M_exit)
        scores = scores.append({"CV":a,"model":"Meta-"+model_exit,"score":meta_model_M_exit.score(Xtest_M_exit,y_test),"k":k},ignore_index=True)
        meta_recall_M_exit = meta_recall_M_exit.append(classwise_recall(k,a,test_prediction_M_exit,y_test.values,"Meta-"+model_exit))       

        # M_exit train test set
        Xtrain_M_int,Xtest_M_int = trainTestData(X_train,X_test,k,model_exit)
        meta_model_M_int = meta_model(Xtrain_M_int,y_train)
        train_prediction_M_int = meta_model_M_int.predict(Xtrain_M_int)
        test_prediction_M_int = meta_model_M_int.predict(Xtest_M_int)
        scores = scores.append({"CV":a,"model":"Meta-"+model_int,"score":meta_model_M_int.score(Xtest_M_int,y_test),"k":k},ignore_index=True)
        meta_recall_M_int = meta_recall_M_int.append(classwise_recall(k,a,test_prediction_M_int,y_test.values,"Meta-"+model_int))       


        #MetaTier
        meta_model_full = meta_model(X_train,y_train)
        meta_full_prediction = meta_model_full.predict(X_test)
        meta2tier_recall = meta2tier_recall.append(classwise_recall(k,a,meta_full_prediction,y_test.values,"Meta2Tier-"+model_int+"-"+model_exit))       
        scores = scores.append({"CV":a,"model":"Meta2Tier-"+model_int+"-"+model_exit,"score":meta_model_full.score(X_test,y_test),"k":k},ignore_index=True)

